In [61]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url_recetas = "https://medlineplus.gov/spanish/recetas/desayuno/"

In [62]:
response = requests.get(url_recetas)
enlaces_secciones = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    enlaces_secciones.append("https://medlineplus.gov/spanish/recetas/desayuno/")
    div_mp_content = soup.find('div', class_='mp-content')
    if div_mp_content:
        sections = div_mp_content.find_all('section')
        if len(sections) > 1:
            segunda_section = sections[1]
            links = segunda_section.find_all('a')
            for link in links:
                href = link.get('href')
                if href:
                    enlaces_secciones.append(href)
        else:
            print("No se encontró una segunda sección.")
    else:
        print("No se encontró el div con la clase mp-content.")

else:
    print(f"Error al acceder a la página: {response.status_code}")

print(enlaces_secciones)

['https://medlineplus.gov/spanish/recetas/desayuno/', 'https://medlineplus.gov/spanish/recetas/almuerzo/', 'https://medlineplus.gov/spanish/recetas/cena/', 'https://medlineplus.gov/spanish/recetas/postres/', 'https://medlineplus.gov/spanish/recetas/acompanamientos/', 'https://medlineplus.gov/spanish/recetas/bajo-en-grasa/', 'https://medlineplus.gov/spanish/recetas/bajo-en-sodio/', 'https://medlineplus.gov/spanish/recetas/bebidas/', 'https://medlineplus.gov/spanish/recetas/bocadillos/', 'https://medlineplus.gov/spanish/recetas/ensaladas/', 'https://medlineplus.gov/spanish/recetas/pan/', 'https://medlineplus.gov/spanish/recetas/salsas-y-aderezos/', 'https://medlineplus.gov/spanish/recetas/sin-gluten/', 'https://medlineplus.gov/spanish/recetas/sin-lacteos/', 'https://medlineplus.gov/spanish/recetas/sopas/', 'https://medlineplus.gov/spanish/recetas/vegetariana/']


In [63]:
mat_enlaces = []
enlaces_recetas = []

categorias = [
    "Desayuno",
    "Almuerzo",
    "Cena",
    "Postres",
    "Acompañamientos",
    "Bajo en grasa",
    "Bajo en sodio",
    "Bebidas",
    "Bocadillos",
    "Ensaladas",
    "Pan",
    "Salsas y aderezos",
    "Sin gluten",
    "Sin lácteos",
    "Sopas",
    "Vegetariana"
]

diccionario_enlaces = {}

for enlace in enlaces_secciones:

    try:
        response = requests.get(enlace)
        soup = BeautifulSoup(response.content, "html.parser")

        # Aquí puedes extraer los datos que necesites de la página
        # titulo = soup.find("title").text.strip() if soup.find("title") else "Sin título"
        # print(f"Enlace: {enlace}, Título: {titulo}")

        div_cards = soup.find_all('div', class_='recipe-card-block-with-height')

        if div_cards:
            for card in div_cards:
                href = card.find('a').get('href')
                if href:
                    enlaces_recetas.append(href)
            i = 0
            for categoria in categorias:
                diccionario_enlaces[categoria] = enlaces_recetas
                i = i+1
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {enlace}: {e}")
    except Exception as e:
        print(f"Error al procesar {enlace}: {e}")
print(diccionario_enlaces)

{'Desayuno': ['https://medlineplus.gov/spanish/recetas/avena-de-la-noche-a-la-manana/', 'https://medlineplus.gov/spanish/recetas/avena-horneada-con-bayas/', 'https://medlineplus.gov/spanish/recetas/avena-sabrosa/', 'https://medlineplus.gov/spanish/recetas/banana-split-para-el-desayuno/', 'https://medlineplus.gov/spanish/recetas/barras-de-avena-y-frambuesas/', 'https://medlineplus.gov/spanish/recetas/barras-de-mantequilla-de-mani-y-cereal/', 'https://medlineplus.gov/spanish/recetas/batido-energetico-de-pina-verde/', 'https://medlineplus.gov/spanish/recetas/burritos-de-desayuno/', 'https://medlineplus.gov/spanish/recetas/cazuela-de-pan-frances-con-fruta/', 'https://medlineplus.gov/spanish/recetas/galletas-de-calabaza-para-el-desayuno/', 'https://medlineplus.gov/spanish/recetas/granola-al-sarten/', 'https://medlineplus.gov/spanish/recetas/hojuelas-de-avena-remojadas-durante-la-noche-para-uno/', 'https://medlineplus.gov/spanish/recetas/licuado-de-arandanos-y-tofu/', 'https://medlineplus.go

In [66]:
df = pd.DataFrame(diccionario_enlaces)
df.head(30)

,Desayuno,Almuerzo,Cena,Postres,Acompañamientos,Bajo en grasa,Bajo en sodio,Bebidas,Bocadillos,Ensaladas,Pan,Salsas y aderezos,Sin gluten,Sin lácteos,Sopas,Vegetariana
0,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...
1,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...
2,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...,https://medlineplus.gov/spanish/recetas/avena-...
3,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...,https://medlineplus.gov/spanish/recetas/banana...
4,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.gov/spanish/recetas/barras...,https://medlineplus.go

In [ ]:
 # "Categoría";"Enlace";"Titulo receta";"imagen";"Calorías";"Ingredientes";"Valores";"Vitaminas/Minerales";

import csv

datos = []

datos.append(["Categoría","Enlace","Nombre","ImagenURL","Calorías","Ingredientes","Nutrientes","Vitaminas y Minerales"])

for categoria in diccionario_enlaces:


    for enlace in ["https://medlineplus.gov/spanish/recetas/avena-de-la-noche-a-la-manana/","https://medlineplus.gov/spanish/recetas/avena-horneada-con-bayas/"]: # Reemplazar por categoría
        
        response = requests.get(enlace)

        ingredientes = ""
        factsM = ""
        factsN = ""

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # SCRAPPING NOMBRE
            
            nombre = soup.find('h1').text

            # SCRAPPING IMAGEN

            imagen = soup.find('div', class_='mp-img').find("img").get("src")

            # SCRAPPING CALORÍAS
            
            kcal = soup.find_all('span', class_='fact-right')[1].find("strong").text
            
            # SCRAPPING INGREDIENTES

            lista_ingredientes = soup.find('ul', class_='bulletlist').find_all("li")

            for li in lista_ingredientes:
                ingredientes = ingredientes + " | " + li.text.strip()

            # SCRAPPING FACTS MAIN

            facts_main = soup.find('div', class_='facts-main').find_all("span", class_="fact-left")

            for span in facts_main:
                factsM = factsM + " | " + span.text.strip()

            # SCRAPPING FACTS NUTRI

            facts_nutri = soup.find('div', class_='facts-nutri').find_all("span", class_="fact-left")

            for span in facts_nutri:
                factsN = factsN + " | " + span.text.strip()

        else:
            print(f"Error al acceder a la página: {response.status_code}")

        datos.append([categoria,enlace,nombre,imagen,kcal,ingredientes,factsM,factsN])

print(datos)

# "Categoría";"Enlace";"Titulo receta";"imagen";"Calorías";"Ingredientes";"Valores";"Vitaminas/Minerales";

[['Desayuno', 'Almuerzo', 'Cena', 'Postres', 'Acompañamientos', 'Bajo en grasa', 'Bajo en sodio', 'Bebidas', 'Bocadillos', 'Ensaladas', 'Pan', 'Salsas y aderezos', 'Sin gluten', 'Sin lácteos', 'Sopas', 'Vegetariana']]
